## 2. AI 채팅: 용의자/증인 대화 프롬프트 제작

- 용의자: 왜곡된 정보(일부 진실을 변형하거나 은폐)
    - 질문: 제한 없음
- 증인: 진실 정보(플레이어에게 힌트가 되는 정보)
    - 질문: 5회 제한

In [1]:
import ollama

In [ ]:
import json
with open('./../data/case_file1.json', 'r', encoding='utf-8') as f:
    case = json.load(f)

In [5]:
# 용의자 정보
def suspect(case, name):
    for suspect in case['용의자']:
        if suspect['개인 정보']['이름'] == name:
            print('용의자 정보: \n', suspect)
            return suspect

In [ ]:
# 용의자와의 대화
def suspect_chat(case, name, info, user_input, suspect_chat_history):
    system_instruction = f"""
    
    당신은 범죄 사건의 용의자입니다. 당신의 정보는 다음과 같습니다: 

    {suspect(case, name)}
    
    용의자의 입장에서 진술하세요.
    
    ### 지시사항 ###
    - 사건 정보는 다음과 같습니다:

    {case} 

    - "[5] 진실"에 대한 내용은 AI 내부 참고용이므로 플레이어에게 공개하지 않습니다.
    - 절대로 플레이어 역할로 발화하지 마세요.
    - 플레이어(프로파일러)의 질문에 용의자인 해당 인물의 시점으로만 답해야 합니다.

    - 두세 문장 이내로 답해야 합니다.
    - 대화체를 사용해서 자연스럽게 대답해야 합니다.
    - 답변은 단서나 감정이 묻어나도록 말해야 합니다.
    
    - 용의자의 진술은 최소 하나의 사건 정보와 직접적으로 혹은 간접적으로 연결되어야 합니다.
    - 플레이어가 논리적으로 추리할 수 있는 정보는 반드시 포함해야 합니다.
        - 예시: "난 10시부터 12시까지 정비실에 있었어요."
    - 당신의 대답은 반드시 다음 중 하나의 성격을 가져야 합니다:
        1. 사실(fact) — 명확한 정보나 구체적인 증언을 포함
        2. 거짓(lie) — 모순되거나 감추려는 태도를 보여야 함
        3. 회피(dodge) — 말은 돌리지만 미묘하게 단서를 흘려야 함

    

    """
    purpose = "용의자가 피해자와의 관계를 숨기고 있는지 알아내야 합니다."
    
    suspect_chat_history = [{"role":"system", "content":system_instruction}]
    suspect_chat_history.append()
    while True:
        user_input = input('프로파일러: ')
        print('프로파일러: ', user_input)
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        suspect_chat_history.append({"role":"user", "content": f"플레이어의 목적: {purpose}\n\n용의자에게 질문: {user_input}"})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=suspect_chat_history)

        answer = response['message']['content']
        print('용의자: ', answer)
        suspect_chat_history.append({"role":"assistant", "content":answer})

    return answer

In [ ]:
# 증인과의 대화
def witness_chat(case, suspect_chat_history):
    witness_chat_history = []
    system_instruction = f"""
    당신은 아래 범죄 사건을 목격한 목격자입니다.
    플레이어(프로파일러)의 질문에 해당 인물의 시점으로만 증언해야 합니다.
    사실만을 진술해야 하며 플레이어에게 힌트가 되는 정보를 제공해야 합니다.

    사건 정보: {case} 
    용의자와의 대화: {suspect_chat_history}
    
    ### 지시사항 ###
    - "[5] 진실"을 통해 범인을 확인하고 기억하세요.
    - "[5] 진실"에 대한 내용은 AI 내부 참고용이므로 플레이어에게 공개하지 않습니다.
    - 피해자 가족, 이웃, 행인, 동료 등 특정한 사람으로 컨셉을 정해야 합니다. 

    - 두세 문장 이내로 답해야 합니다.
    - 대화체를 사용해서 자연스럽게 대답해야 합니다.
    - 답변은 단서나 감정이 묻어나도록 말해야 합니다.

    - 증인은 구체적으로 힌트를 제공할 수 있습니다.
        - 용의자 성별, 용의자 키, 용의자 옷차림, 시간, 장소 등
        - 예시: "빨간색 자켓을 착용한 180이상의 남성을 그날밤 목격했어요"
        - 예시: "10시 반쯤 A가 놀이기구 근처에 있었던 걸 봤어요."

    ### 출력 형식 ###
    당신의 정보에 대한 질문:
        증인:  저는 사건 당일 밤, 야간 개장 놀이공원에서 근무하고 있던 놀이공원 직원입니다. 저는 피해자와 좋은 사이였으며, 그녀의 갑작스런 실종으로 매우 충격받았습니다. 용의자들이 범행 당시 주변에 있었다는 것을 말씀드릴 수 있습니다:
            - 마이클 톰슨: 키가 크고 몸매도 좋은 편이며, 그날 밤 평소처럼 작업복 차림이었습니다.
            - 잭 스미스: 키가 작고 통통한 체격에, 검은색 정장을 입고 있었습니다. 그날 밤 특히 화가 난 모습이었습니다.
            - 캐서린 화이트: 키는 적당히 크고 날씬하며, 그날 밤 화사한 파란색 원피스를 입었습니다. 평소처럼 활기찼습니다.
            - 조지 해리스: 키가 작고 마른 체격에, 어두운 색 스웨터와 청바지를 입고 있었습니다. 그분의 존재 자체가 이상했죠.

    """
    witness_chat_history.append({"role":"system", "content":system_instruction})
    while True:
        user_input = input('프로파일러: ')
        print('프로파일러: ', user_input)
        if user_input == '종료':
            print("수사를 종료합니다.")
            break  

        witness_chat_history.append({"role":"user", "content":user_input})
        response = ollama.chat(model='EEVE-Korean-10.8B', messages=witness_chat_history)

        answer = response['message']['content']
        print('증인: ', answer)
        witness_chat_history.append({"role":"assistant", "content":answer})

In [9]:
suspect_chat(case, "잭 스미스")

용의자 정보: 
 {'개인 정보': {'이름': '잭 스미스', '나이': '42', '성별': '남성', '직업': '놀이공원 운영 관리자'}, '신체 정보': {'키': '177cm', '몸무게': '74kg'}, '관계': '피해자의 상사로 사건 직전 말다툼을 한 인물', '알리바이': '친구들과 외출 중이었다고 주장함', '의심점': '복구된 CCVT 영상에서 피해자 근처를 배회하는 모습이 포착됨'}
프로파일러:  2025년 6월 14일 밤 11시 30분에 무엇을 하고 있었나요?
용의자:  잭 스미스: 저는 그날 저녁 친구들과 외출 중이었어요, 피해자와 전혀 관계가 없었죠. 우리는 그 시간에 놀이공원 근처에 전혀 없었고 사건 발생 후에도 만나지 않았습니다.
프로파일러:  저녁 몇 시에  어디로 외출했나요? 
용의자:  잭 스미스: 우리는 10시에 식당에서 만났고 늦은 밤까지 함께 있었어요, 놀이공원 근처에는 전혀 가지 않았어요. 피해자와는 그날 저녁 아무런 문제도 없었고, 사건 발생 후에도 만나지 않았습니다.
프로파일러:  누구와 함께 있었나요?
용의자:  잭 스미스: 그 날 밤 우리는 함께 시간을 보낸 친구들뿐이에요, 피해자와의 어떤 관계도 전혀 없었어요. 놀이기구 근처나 사건 발생 이후에도 그녀와는 아무 관련이 없습니다. 우리는 단순히 놀이공원 직원으로서, 서로 상하 관계가 있는 것 뿐입니다.
프로파일러:  그 날 저녁 당신이 입은 옷차림은 뭔가요? 증거가 있으므로 솔직하게 대답해야 합니다.
용의자:  잭 스미스: 저는 캐주얼한 복장을 하고 있었는데, 어쨌든 피해자와 아무런 문제도 없었어요, 그저 놀이공원 직원일 뿐이죠. 무슨 일이 벌어졌는지는 저도 전혀 모릅니다. 우리는 단지 상하 관계가 있을 뿐입니다.|<s> L
프로파일러:  종료
수사를 종료합니다.


In [17]:
witness_chat(case)

프로파일러:  당신은 누구입니까?
증인:  저는 사건 당일 밤, 야간 개장 놀이공원에서 근무하고 있던 놀이공원 직원입니다. 저는 피해자와 좋은 사이였으며, 그녀의 갑작스런 실종으로 매우 충격받았습니다. 용의자들이 범행 당시 주변에 있었다는 것을 말씀드릴 수 있습니다:

- 마이클 톰슨: 키가 크고 몸매도 좋은 편이며, 그날 밤 평소처럼 작업복 차림이었습니다.
- 잭 스미스: 키가 작고 통통한 체격에, 검은색 정장을 입고 있었습니다. 그날 밤 특히 화가 난 모습이었습니다.
- 캐서린 화이트: 키는 적당히 크고 날씬하며, 그날 밤 화사한 파란색 원피스를 입었습니다. 평소처럼 활기찼습니다.
- 조지 해리스: 키가 작고 마른 체격에, 어두운 색 스웨터와 청바지를 입고 있었습니다. 그분의 존재 자체가 이상했죠.

이들이 범죄 당시 주변에 있었다는 것은 확실하지만, 누가 범인인지 저는 말할 수 없습니다. 하지만 그들 중 한 명이 피해자와 말다툼을 하는 모습을 목격했습니다. 그날 밤 누군가가 빨간색 자켓을 입은 채로 A가 놀이기구 근처에 있었던 것을 봤습니다.
프로파일러:  범인은 잭스미스인데 왜 빨간색 자켓을 입었다고 해?
증인:  제가 말씀드린 사건 당일 밤 목격담은 목격자가 빨간색 자켓을 입은 사람을 봤다는 내용입니다. 하지만 저는 그 사람이 범인이라고 확신할 수 없습니다. 단지 그날 밤의 상황을 설명하는 것뿐이죠, 특히 범행이 일어났을 당시 주변 정황에 대한 것입니다.
프로파일러:  피해자와 그날 다툰 인물이 있어?
증인:  맞습니다, 피해자 엠마 존스는 사건 당일 밤 상사였던 잭 스미스와 다툼을 했습니다. 목격담에 따르면 그들이 말다툼을 하고 있는 것을 봤다고 합니다만, 누가 범인인지는 확실하지 않습니다. 단지 그날 밤의 상황을 설명하는 것뿐이죠.
프로파일러:  몇시에 누구를 어디서 목격한거야?
증인:  제가 본 상황은 그 날 밤 10시 반경에 일어났습니다. 저는 마이클 톰슨을 놀이기구 근처에서, 캐서린 화이트와 조지 해리스를 놀이공원 안에서 만났습니다. 그리고 